### Comment:
It was fun to do the Klotski challenge, but it's more efficient to just brute force Klotski. It took me less than half a second to find all 25955 possible positions (counting every similar block as equal). The shortest solution has 116 single steps and every configuration can be reached in 137 steps.

With recursion and backtracking, I got my solution in 2 seconds, but it required 3873 single steps (or 20928 steps when I forgot counting every similar block as equal).

In [1]:
import operator
import numpy as np

In [2]:
pieces = [[[0, 3], [0, 4]], 
          [[1, 3], [2, 3], [1, 4], [2, 4]],
          [[3, 3], [3, 4]],
          [[0, 1], [0, 2]],
          [[1, 2], [2, 2]],
          [[3, 1], [3, 2]],
          [[1, 1]],
          [[2, 1]],
          [[0, 0]],
          [[3, 0]]]

easy_pieces = [[[0, 3], [0, 4]],
               [[1, 1], [2, 1], [1, 2], [2, 2]],
               [[3, 3], [3, 4]],
               [[0, 1], [0, 2]],
               [[1, 4], [2, 4]],
               [[3, 1], [3, 2]],
               [[1, 3]],
               [[2, 3]],
               [[1, 0]],
               [[2, 0]]]

In [3]:
class Board:
    def __init__(self, board_state):
        self.board_state = list(board_state)
        self.children = []
        
    def check_solved(self):
        if self.board_state[1] == [[1, 0], [2, 0], [1, 1], [2, 1]]:
            return True
        return False
    
    def move(self, piece, direction):
        return [list(map(operator.add, point, direction)) for point in self.board_state[piece]]
    
    def move_possible(self, piece, direction):
        possible_piece = self.move(piece, direction)
        # außerhalb des Feldes
        if (True in [x_coordinate > 3 or x_coordinate < 0 for 
                     x_coordinate in [point[0] for point in possible_piece]] or 
            True in [y_coordinate > 4 or y_coordinate < 0 for 
                     y_coordinate in [point[1] for point in possible_piece]]):
            return False
        # Überlappung
        for point in possible_piece:
            if point in self.board_state[piece]:
                pass
            elif 1 in [block.count(point) for block in self.board_state]:
                return False
        return True
    
    def add_child(self, obj):
        self.children.append(obj)

In [4]:
def generate_children(board):
    for iter_piece in range(10):
        for direction in [[0, 1], [0, -1], [1, 0], [-1, 0]]:
            if board.move_possible(iter_piece, direction): 
                child_state = list(board.board_state)
                child_state[iter_piece] = board.move(iter_piece, direction)
                board.add_child(Board(child_state))

In [6]:
def build_and_chack_tree(depth, root):
    root = [root]
    current_layer = []
    history = []
    for n in range(depth):
        for child in root:
            generate_children(child)
        for node in root:
            for child in node.children:
                if not compress(child.board_state) in history:
                    current_layer.append(child)
                    history.append(compress(child.board_state))
        print(n, len(current_layer))
        if any([board.check_solved() for board in current_layer]):
            break
        root = list(current_layer)
        current_layer = []
    return current_layer

In [7]:
def compress(input_pieces):
    len1 = []
    len2 = []
    len4 = []
    for i in range(len(input_pieces)):
        if len(input_pieces[i]) == 1:
            len1.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len1.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 2:
            len2.append((input_pieces[i][0][0], input_pieces[i][0][1]))
            len2.sort(key = operator.itemgetter(0, 1))
        if len(input_pieces[i]) == 4:
            len4.append((input_pieces[i][0][0], input_pieces[i][0][1]))
    return [len1, len2, len4]

In [8]:
def draw_board(piece):
    board = np.full((5, 4), ' ')
    for i in range(10):
        # X-Wert
        for x in range(len(piece[i])):
            board[piece[i][x][1]][piece[i][x][0]] = i
    print(np.flip(np.fliplr(board)), "\n")

In [9]:
for i in build_and_chack_tree(300, Board(pieces)):
    if i.check_solved():
        draw_board(i.board_state)

0 4
1 11
2 13
3 18
4 16
5 19
6 20
7 22
8 32
9 38
10 42
11 36
12 34
13 42
14 40
15 34
16 30
17 29
18 36
19 47
20 60
21 54
22 48
23 48
24 39
25 35
26 45
27 52
28 68
29 74
30 69
31 51
32 50
33 65
34 77
35 99
36 132
37 145
38 171
39 201
40 238
41 282
42 342
43 376
44 393
45 426
46 403
47 402
48 440
49 477
50 538
51 568
52 566
53 608
54 686
55 741
56 751
57 720
58 707
59 640
60 535
61 489
62 479
63 472
64 472
65 403
66 311
67 292
68 270
69 227
70 189
71 167
72 156
73 130
74 94
75 70
76 59
77 50
78 45
79 50
80 44
81 58
82 70
83 88
84 102
85 110
86 106
87 120
88 136
89 150
90 178
91 190
92 216
93 236
94 256
95 262
96 285
97 312
98 317
99 336
100 331
101 286
102 249
103 225
104 215
105 208
106 213
107 195
108 178
109 187
110 174
111 143
112 104
113 89
114 79
115 73
[['5' '3' '2' '0']
 ['5' '3' '2' '0']
 ['6' '9' '4' '4']
 [' ' '1' '1' '7']
 [' ' '1' '1' '8']] 

[['2' '0' '5' '3']
 ['2' '0' '5' '3']
 ['4' '4' '8' '7']
 ['6' '1' '1' ' ']
 ['9' '1' '1' ' ']] 

